#### In Lecture 05 we looked a scipy odeint

Here we look at other Scipy builtin functions for ODE solvers.

https://docs.scipy.org/doc/scipy/reference/integrate.html

```
solve_ivp
```
is a newer form of odeint

In [5]:
!which python

/Users/chrishill/projects/osn/miniconda3/envs/py38/bin/python


In [9]:
import sys
import numpy as np
from scipy.integrate import odeint, solve_ivp
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from IPython.display import clear_output
from IPython.display import Video
from celluloid import Camera
from IPython.display import HTML

In [83]:
# Pendulum rod lengths (m), bob masses (kg).
L1, L2 = 1., 1.
m1, m2 = 1., 1.
# The gravitational acceleration (m.s-2).
g = 9.81

def deriv(t, y, L1, L2, m1, m2):
# def deriv(y, t, L1, L2, m1, m2):
    """Return the first derivatives of y = theta1, z1, theta2, z2."""
    theta1, z1, theta2, z2 = y

    c, s = np.cos(theta1-theta2), np.sin(theta1-theta2)

    theta1dot = z1
    z1dot = (m2*g*np.sin(theta2)*c - m2*s*(L1*z1**2*c + L2*z2**2) -
             (m1+m2)*g*np.sin(theta1)) / L1 / (m1 + m2*s**2)
    theta2dot = z2
    z2dot = ((m1+m2)*(L1*z1**2*s - g*np.sin(theta2) + g*np.sin(theta1)*c) + 
             m2*L2*z2**2*s*c) / L2 / (m1 + m2*s**2)
    return theta1dot, z1dot, theta2dot, z2dot

def calc_E(y):
    """Return the total energy of the system."""

    th1, th1d, th2, th2d = y.T
    V = -(m1+m2)*L1*g*np.cos(th1) - m2*L2*g*np.cos(th2)
    T = 0.5*m1*(L1*th1d)**2 + 0.5*m2*((L1*th1d)**2 + (L2*th2d)**2 +
            2*L1*L2*th1d*th2d*np.cos(th1-th2))
    return T + V

# Maximum time, time point spacings and the time grid (all in s).
tmax, dt = 1., 0.01
t = np.arange(0, tmax+dt, dt)
# Initial conditions: theta1, dtheta1/dt, theta2, dtheta2/dt.
y0 = np.array([3*np.pi/7, 0, 3*np.pi/4, 0])

# Do the numerical integration of the equations of motion
# y = odeint(deriv, y0, t, args=(L1, L2, m1, m2))

# Do same with solve_ivp
yy = solve_ivp(deriv,(0.,1.,),y0,t_eval=t,args=(L1,L2,m1,m2,),rtol=1.e-9)
y=yy.y.T

# Check that the calculation conserves total energy to within some tolerance.
EDRIFT = 0.05
# Total energy from the initial conditions
E = calc_E(y0)
if np.max(np.sum(np.abs(calc_E(y) - E))) > EDRIFT:
    sys.exit('Maximum energy drift of {} exceeded.'.format(EDRIFT))

# Unpack z and theta as a function of time
theta1, theta2 = y[:,0], y[:,2]

# Convert to Cartesian coordinates of the two bob positions.
x1 = L1 * np.sin(theta1)
y1 = -L1 * np.cos(theta1)
x2 = x1 + L2 * np.sin(theta2)
y2 = y1 - L2 * np.cos(theta2)

# Plotted bob circle radius
r = 0.05
# Plot a trail of the m2 bob's position for the last trail_secs seconds.
trail_secs = 1
# This corresponds to max_trail time points.
max_trail = int(trail_secs / dt)

def make_plot(i):
    # Plot and save an image of the double pendulum configuration for time
    # point i.
    # The pendulum rods.
    ax.plot([0, x1[i], x2[i]], [0, y1[i], y2[i]], lw=2, c='k')
    # Circles representing the anchor point of rod 1, and bobs 1 and 2.
    c0 = Circle((0, 0), r/2, fc='k', zorder=10)
    c1 = Circle((x1[i], y1[i]), r, fc='b', ec='b', zorder=10)
    c2 = Circle((x2[i], y2[i]), r, fc='r', ec='r', zorder=10)
    ax.add_patch(c0)
    ax.add_patch(c1)
    ax.add_patch(c2)

    # The trail will be divided into ns segments and plotted as a fading line.
    ns = 20
    s = max_trail // ns

    for j in range(ns):
        imin = i - (ns-j)*s
        if imin < 0:
            continue
        imax = imin + s + 1
        # The fading looks better if we square the fractional length along the
        # trail.
        alpha = (j/ns)**2
        ax.plot(x2[imin:imax], y2[imin:imax], c='r', solid_capstyle='butt',
                lw=2, alpha=alpha)

    # Centre the image on the fixed anchor point, and ensure the axes are equal
    ax.set_xlim(-L1-L2-r, L1+L2+r)
    ax.set_ylim(-L1-L2-r, L1+L2+r)
    ax.set_aspect('equal', adjustable='box')
    plt.axis('off')
    # plt.savefig('frames/_img{:04d}.png'.format(i//di), dpi=72)
    # plt.cla()


# Make an image every di time points, corresponding to a frame rate of fps
# frames per second.
# Frame rate, s-1
fps = 10
di = int(1/fps/dt)
fig = plt.figure(figsize=(8.3333, 6.25), dpi=72)
ax = fig.add_subplot(111)

camera=Camera(fig)

for i in range(0, t.size, di):
    print(i // di, '/', t.size // di)
    make_plot(i)
    camera.snap()

animation=camera.animate()
plt.close()
animation.save('pendu_animation.mp4',fps=40)
Video("pendu_animation.mp4")

0 / 10
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


In [80]:
np.shape(yy.y.T)

(101, 4)

In [75]:
yodeint

array([[ 1.34639685e+00,  0.00000000e+00,  2.35619449e+00,
         0.00000000e+00],
       [ 1.34594732e+00, -8.98990453e-02,  2.35608673e+00,
        -2.15719903e-02],
       [ 1.34459915e+00, -1.79714776e-01,  2.35576222e+00,
        -4.33920757e-02],
       [ 1.34235358e+00, -2.69364010e-01,  2.35521724e+00,
        -6.57078416e-02],
       [ 1.33921270e+00, -3.58763857e-01,  2.35444559e+00,
        -8.87657543e-02],
       [ 1.33517941e+00, -4.47831821e-01,  2.35343864e+00,
        -1.12810639e-01],
       [ 1.33025744e+00, -5.36485886e-01,  2.35218528e+00,
        -1.38085124e-01],
       [ 1.32445134e+00, -6.24644556e-01,  2.35067203e+00,
        -1.64829089e-01],
       [ 1.31776647e+00, -7.12226851e-01,  2.34888301e+00,
        -1.93279098e-01],
       [ 1.31020900e+00, -7.99152233e-01,  2.34679999e+00,
        -2.23667840e-01],
       [ 1.30178589e+00, -8.85340466e-01,  2.34440243e+00,
        -2.56223553e-01],
       [ 1.29250491e+00, -9.70711405e-01,  2.34166755e+00,
      

In [44]:
y.y

array([[ 1.34639685e+00,  1.34594732e+00,  1.34459917e+00,
         1.34235392e+00,  1.33921363e+00,  1.33518104e+00,
         1.33025975e+00,  1.32445420e+00,  1.31776965e+00,
         1.31021222e+00,  1.30178884e+00,  1.29250730e+00,
         1.28237621e+00,  1.27140504e+00,  1.25960407e+00,
         1.24698443e+00,  1.23355810e+00,  1.21933788e+00,
         1.20434883e+00,  1.18861023e+00,  1.17212995e+00,
         1.15491846e+00,  1.13698883e+00,  1.11835670e+00,
         1.09904034e+00,  1.07906057e+00,  1.05844084e+00,
         1.03720718e+00,  1.01538822e+00,  9.93015166e-01,
         9.70121837e-01,  9.46744636e-01,  9.22922563e-01,
         8.98697211e-01,  8.74112766e-01,  8.49216008e-01,
         8.24056311e-01,  7.98685642e-01,  7.73158563e-01,
         7.47532229e-01,  7.21865166e-01,  6.96213842e-01,
         6.70656093e-01,  6.45275992e-01,  6.20160067e-01,
         5.95397306e-01,  5.71079156e-01,  5.47299523e-01,
         5.24154769e-01,  5.01743717e-01,  4.80167647e-0

In [31]:
[*(L1,L2,m1,m2,)]

[1.0, 1.0, 1.0, 1.0]

In [30]:
a

1.0

In [33]:
a,b,c,d=y0

In [34]:
a

1.3463968515384828

In [35]:
y0

array([1.34639685, 0.        , 2.35619449, 0.        ])

In [40]:
?solve_ivp

Signature:
solve_ivp(
    fun,
    t_span,
    y0,
    method='RK45',
    t_eval=None,
    dense_output=False,
    events=None,
    vectorized=False,
    args=None,
    **options,
)
Docstring:
Solve an initial value problem for a system of ODEs.

This function numerically integrates a system of ordinary differential
equations given an initial value::

    dy / dt = f(t, y)
    y(t0) = y0

Here t is a 1-D independent variable (time), y(t) is an
N-D vector-valued function (state), and an N-D
vector-valued function f(t, y) determines the differential equations.
The goal is to find y(t) approximately satisfying the differential
equations, given an initial value y(t0)=y0.

Some of the solvers support integration in the complex domain, but note
that for stiff ODE solvers, the right-hand side must be
complex-differentiable (satisfy Cauchy-Riemann equations [11]_).
To solve a problem in the complex domain, pass y0 with a complex data type.
Another option always available is to rewrite your pro